# Minimax en Python: le jeu de Nim

adapted from: Geir Arne Hjelle https://realpython.com/python-minimax-nim/ 

avec l'aide de ChatGPT pour la traduction.

Erreurs dûes à Marc Lelarge, merci de les signaler: https://github.com/mlelarge/agreg/issues

[Nim](https://fr.wikipedia.org/wiki/Jeux_de_Nim) est un jeu pour deux joueurs qui se termine toujours avec la victoire d'un joueur. Le jeu consiste en plusieurs jetons posés sur la table de jeu et les joueurs qui se relaient pour en retirer un ou plusieurs. Dans la première partie, nous étudions une version simplifiée de Nim avec les règles suivantes :

- Il y a plusieurs jetons dans un tas commun.
- Deux joueurs se relaient.
- A son tour, un joueur retire un, deux ou trois jetons du tas.
- Le joueur qui prend le dernier jeton perd la partie.

Nous allons appeler ce jeu Simple-Nim. Plus tard, nous allons considérer les vraies règles de Nim. Elles ne sont pas beaucoup plus compliquées, mais Simple-Nim est plus facile à présenter.

Voici un exemple de jeux:

| Mindy | Tas       | Maximillien |
|:-------|:------------:|------------:|
|       | 🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙 |            |
| 🪙🪙      | 🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙 |            |
|       | 🪙🪙🪙🪙🪙🪙🪙🪙🪙 |      🪙      |
|   🪙🪙🪙    | 🪙🪙🪙🪙🪙🪙 |          |
|      | 🪙🪙🪙🪙 |     🪙🪙     |
|   🪙🪙🪙   | 🪙 |         |
|      |  |     🪙    |

Vous pouvez représenter le même jeu en ne gardant en compte que le nombre de jetons dans le tas et à qui c'est le tour :

| Tour | Tas   | 
|:-------|:------------|
|   Mindy    | 🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙 (12)| 
| Maximillien    | 🪙🪙🪙🪙🪙🪙🪙🪙🪙🪙 (10) |
|   Mindy    | 🪙🪙🪙🪙🪙🪙🪙🪙🪙 (9)|
|   Maximillien   | 🪙🪙🪙🪙🪙🪙 (7)|
|    Mindy  | 🪙🪙🪙🪙 (4)|
|   Maximillien   | 🪙 (1) |

On peut donc rerésenter le jeux par: **12-10-9-6-4-1, Mindy commence**, ce sera l'état du jeux (game state).
En partant d'une pile de six jetons, il y a seulement treize jeux différents possibles qui peuvent être joués:
![](./images/minmax_nim_tree.png)

## Stratégie optimale

En fait, Simple-Nim est un peu trop simple: en y regardant un peu on s'apercoit rapidement que le premier joueur gagne dès que le tas a une hauteur $k$ avec $k \text{ mod } 4 \neq 1$. En effet, si $k \text{ mod } 4 = 1$ alors quelque soit le premier coup joué par Maximilian, Mindy peut toujours garantir que $k'\text{ mod } 4 = 1$ au prochain coup de Maximilian. Donc en fin de partie, Maximilian a un tas de taille 1 et perd. Par contre si $k \text{ mod } 4 \neq 1$, le même argument montre que Maximilian peut toujours garantir que $k' \text{ mod } 4 =1$ pour son adversaire tout au long de la partie. Le jeu est donc résolu! Cependant il est pratique pour présenter l'algorithme minimax qui sera ensuite étendu à des cas plus difficiles...

La base de l'algorithme minimax: vous donnez à chacun des deux joueurs le rôle de joueur maximisant ou minimisant. Le joueur actuel veut faire un coup pour maximiser ses chances de gagner, tandis que son adversaire veut contrer avec un coup pour minimiser les chances de victoire du joueur actuel.

Pour suivre le jeu, dessinez l'arbre de tous les coups possibles. Ensuite, attribuez un score minimax à tous les nœuds feuilles de l'arbre, i.e..les nœuds avec un compteur à zéro. Le score dépendra de l'issue représentée par le nœud feuille: si le joueur qui maximise a gagné la partie, la feuille a un score de +1. De même, si le joueur minimisant a gagné la partie, la feuille a un score de -1:
![](./images/minmax_nim_tree_leaves.png)

Les nœuds feuilles dans les rangées marquées Max — Maximillian, le joueur qui maximise — sont marqués par +1, tandis que les nœuds feuilles dans les rangées de Mindy sont marqués par -1. Ensuite, laissez les scores minimax monter dans l'arbre. Considérez un nœud où tous les enfants ont reçu un score. Si le nœud est sur une ligne Max, donnez-lui le score maximum de ses enfants. Sinon, donnez-lui le score minimum de ses enfants.
![](./images/minmax_nim_tree_all.png)

## Code python pour la version simple

Voici une version très simple de l'algorithme:

In [1]:
def minimax(state, max_turn):
    # renvoie 1 si Maximilien gagne la partie et -1 s'il perd
    if state == 0:
        return 1 if max_turn else -1
    
    # nouveaux etats possibles
    possible_new_states = [
        state - take for take in (1, 2, 3) if take <= state
    ]
    
    # algorithem minmax
    if max_turn:
        scores = [
            minimax(new_state, max_turn=False)
            for new_state in possible_new_states
        ]
        return max(scores)
    else:
        scores = [
            minimax(new_state, max_turn=True)
            for new_state in possible_new_states
        ]
        return min(scores)

In [2]:
minimax(6, True)

1

In [3]:
minimax(5, False)

1

In [4]:
minimax(4, False)

-1

Pour trouver efficacement quel coup Maximilien doit jouer ensuite, vous pouvez faire les calculs en boucle :

In [5]:
state = 6
for take in (1, 2, 3):
    new_state = state - take
    score = minimax(new_state, max_turn=False)
    print(f"Move from {state} to {new_state}: {score}")

Move from 6 to 5: 1
Move from 6 to 4: -1
Move from 6 to 3: -1


In [6]:
def best_move(state):
    for take in (1, 2, 3):
        new_state = state - take
        score = minimax(new_state, max_turn=False)
        if score > 0:
            break
    return score, new_state

In [7]:
best_move(6)

(1, 5)

In [8]:
best_move(5)

(-1, 2)

## Refactoring

Les deux fonctions `minimax()` et `best_move()` contiennent une logique qui traite de l'algorithme minimax et une logique qui traite des règles de Simple-Nim. Nous allons voir comment les séparer.

Nous commencons par les règles de Simple-Nim:

In [9]:
def possible_new_states(state):
    return [state - take for take in (1, 2, 3) if take <= state]

def evaluate(state, is_maximizing):
    if state == 0:
        return 1 if is_maximizing else -1

On peut maintenant reprendre le code de l'algorithme minimax:

In [10]:
def minimax(state, is_maximizing):
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    if is_maximizing:
        scores = [
            minimax(new_state, is_maximizing=False)
            for new_state in possible_new_states(state)
        ]
        return max(scores)
    else:
        scores = [
            minimax(new_state, is_maximizing=True)
            for new_state in possible_new_states(state)
        ]
        return min(scores)

Remarque dans la première ligne, on utilise `:=` [Assignment Expressions](https://peps.python.org/pep-0572/)

Ensuite, observez que les blocs de l'instruction `if … else` sont assez similaires. Les seules différences entre les blocs sont la fonction, `max()` ou `min()`, utilisée pour trouver le meilleur score et la valeur pour `is_maximizing` dans les appels récursifs à `minimax()`. Ces deux éléments peuvent être directement calculés à partir de la valeur actuelle de `is_maximizing`.

In [11]:
def minimax(state, is_maximizing):
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    return (max if is_maximizing else min)(
        minimax(new_state, is_maximizing=not is_maximizing)
        for new_state in possible_new_states(state)
    )

Notez qu'on utilise la fonction [`max`](https://docs.python.org/3/library/functions.html#max) (ou `min`) appliquée à un itérateur.

les règles de Simple-Nim ne sont pas explicitement encodées dans l'algorithme `minimax`. Au lieu de cela, ils sont encapsulés dans `possible_new_states()` et `evaluate()`.

In [12]:
minimax(6, is_maximizing=True)

1

In [13]:
minimax(5, is_maximizing=False)

1

In [14]:
minimax(4, is_maximizing=False)

-1

On peut maintenant modifier `best_move`:

In [15]:
def best_move(state):
    return max(
        (minimax(new_state, is_maximizing=False), new_state)
        for new_state in possible_new_states(state)
    )

Comme précédemment, vous considérez (et renvoyez) un tuple contenant le score et le meilleur nouvel état. Étant donné que les comparaisons, y compris `max()`, sont effectuées élément par élément dans les tuples, le score doit être le premier élément du tuple.

In [16]:
best_move(6)

(1, 5)

## Mémoïsation

Un défi avec l'algorithme minimax est que les arbres de jeu peuvent être énormes.

Dans Simple-Nim, l'arbre de jeu se compose de nombreux états de jeu répétés. Par exemple, vous pouvez passer de six à trois compteurs de quatre manières différentes : 6-3, 6-4-3, 6-5-3 et 6-5-4-3. Par conséquent, les mêmes états de jeu sont calculés à plusieurs reprises par `minimax()`. Vous pouvez contrer cela en utilisant un `cache`:

In [17]:
from functools import cache

def possible_new_states(state):
    return [state - take for take in (1, 2, 3) if take <= state]

def evaluate(state, is_maximizing):
    if state == 0:
        return 1 if is_maximizing else -1
    
def minimax(state, is_maximizing):
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    return (max if is_maximizing else min)(
        minimax(new_state, is_maximizing=not is_maximizing)
        for new_state in possible_new_states(state)
    )

@cache
def minimax_cache(state, is_maximizing):
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    return (max if is_maximizing else min)(
        minimax_cache(new_state, is_maximizing=not is_maximizing)
        for new_state in possible_new_states(state)
    )

In [18]:
%timeit minimax(25, True)

3.7 s ± 516 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit minimax_cache(25, True)

95 ns ± 2.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Variantes du jeu de Nim

Il est temps de regarder les règles habituelles de Nim. Vous pouvez toujours reconnaître le jeu, mais il laisse un peu plus de choix aux joueurs :

- Il y a plusieurs piles, avec un certain nombre de jetons dans chacune.
- Deux joueurs jouent à tour de rôle.
- A son tour, un joueur retire autant de pions qu'il le souhaite, mais les pions doivent provenir de la même pile.
- Le joueur qui prend le dernier compteur perd la partie.

Notez qu'il n'y a plus de restriction sur le nombre de marqueurs à retirer par tour. Si une pile contient vingt marqueurs, alors le joueur actif peut tous les prendre.

Adaptez le code!

In [20]:
def possible_new_states(state):
    for pile, counters in enumerate(state):
        for remain in range(counters):
            yield state[:pile] + (remain,) + state[pile + 1 :]

On représente l'état du jeu sous la forme d'un tuple de nombres, chaque nombre représentant le nombre de jetons dans la pile correspondante.
Pour chaque tas, nous utilisons `range()` pour répertorier tous les choix possibles du nombre de tokens pouvant rester dans ce tas. On retourne un nouvel état du jeu en copiant l'état à l'exception du tas actuel. Notez que les tuples peuvent être découpés avec des crochets `([])` et concaténés avec le signe plus `(+)`.

In [21]:
def evaluate(state, is_maximizing):
    if all(counters == 0 for counters in state):
        return 1 if is_maximizing else -1

In [22]:
best_move((2, 3, 5))

(1, (2, 3, 1))

In [23]:
minimax((2,3,5), True)

1

Nim est parfois qualifié de jeu [misère](https://en.wikipedia.org/wiki/Mis%C3%A8re) car le but est d'éviter de prendre le dernier contre. Une variante populaire de Nim modifie la condition de victoire. Dans cette variante, le joueur qui prend le dernier marqueur remporte la partie. Comment changeriez-vous votre code pour jouer à cette version du jeu ?

In [24]:
def evaluate(state, is_maximizing):
    if all(counters == 0 for counters in state):
        return -1 if is_maximizing else 1

In [25]:
best_move((2, 3, 5))

(1, (2, 3, 1))

In [26]:
minimax((2,3,5), True)

1

Une autre variante de Nim commence avec tous les pions en une seule pile :

- Il y a plusieurs pions, tous commençant par une seule pile.
- Deux joueurs jouent à tour de rôle.
- À son tour, un joueur divise une pile en deux, de sorte que les deux nouvelles piles aient des nombres de jetons différents.
- Le premier joueur qui ne peut diviser aucune pile perd la partie.

Dans cette variante, chaque mouvement crée une nouvelle pile. Le jeu dure jusqu'à ce que toutes les piles contiennent un ou deux jetons, car ces piles ne peuvent pas être divisées.

In [27]:
def possible_new_states(state):
    for pile, counters in enumerate(state):
        for take in range(1, (counters + 1) // 2):
            yield state[:pile] + (counters - take, take) + state[pile + 1 :]

def evaluate(state, is_maximizing):
    if all(counters <= 2 for counters in state):
        return -1 if is_maximizing else 1

## Élagage alpha-bêta

Revenons maintenant à l'exemple où c'est au tour de Maximilien avec six jetons sur la table. Si vous considérez les branches de gauche à droite et arrêtez d'explorer les sous-arbres une fois que le score minimax d'un nœud est décidé, vous vous retrouverez avec l'arbre suivant :
![](./images/minmax_nim_tree_alphabeta.png)

Vous avez besoin d'un critère pour savoir quand vous pouvez arrêter d'explorer. Pour ce faire, vous allez ajouter deux paramètres, alpha et beta :

- alpha représentera le score minimum que le joueur maximisant est assuré.
- beta représentera le score maximum que le joueur minimisant est assuré.

Si beta est inférieure ou égale à alpha, le joueur peut arrêter d'explorer cet arbre de jeu. La maximisation aura déjà trouvé une meilleure option que celle que le joueur pourrait trouver en explorant plus avant.

Pour implémenter cette idée, vous commencerez par remplacer votre compréhension par une boucle for explicite. Vous avez besoin de la boucle explicite pour pouvoir en sortir et élaguer efficacement l'arbre...

In [28]:
def best_move(state):
    return max(
        (minimax(new_state, is_maximizing=False), new_state)
        for new_state in possible_new_states(state)
    )

def possible_new_states(state):
    for pile, counters in enumerate(state):
        for remain in range(counters):
            yield state[:pile] + (remain,) + state[pile + 1 :]

def evaluate(state, is_maximizing):
    if all(counters == 0 for counters in state):
        return 1 if is_maximizing else -1   
    
@cache
def minimax(state, is_maximizing, alpha=-1, beta=1):
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    scores = []
    for new_state in possible_new_states(state):
        #scores.append(
        #    score := minimax(new_state, not is_maximizing, alpha, beta)
        #)
        score = minimax(new_state, not is_maximizing, alpha, beta)
        if is_maximizing:
            alpha = max(alpha, score)
        else:
            beta = min(beta, score)
        if beta <= alpha:
            break
    return alpha if is_maximizing else beta #(max if is_maximizing else min)(scores)

In [29]:
minimax((4,4), True)

-1

In [30]:
%timeit minimax((2,5,6,6,5), True)

120 ns ± 8.94 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [31]:
minimax((2,5,6,6,5), True)

1

In [32]:
best_move((2,5,6,6,5))

(1, (2, 5, 6, 4, 5))